In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf

asset = "SPY"
file_train_x = f"./output/{asset}/DATA/{asset}-60-3/train_x.csv"
file_train_y = f"./output/{asset}/DATA/{asset}-60-3/train_y.csv"
file_val_x = f"./output/{asset}/DATA/{asset}-60-3/val_x.csv"
file_val_y = f"./output/{asset}/DATA/{asset}-60-3/val_y.csv"


# filenames = [file1, file2]
# record_defaults1 = [tf.float32] * 54*60
# record_defaults2 = [tf.float32] * 1
# dataset_train_x = tf.contrib.data.CsvDataset(file1,record_defaults1)
# dataset_train_y = tf.contrib.data.CsvDataset(file2,record_defaults2)
# dataset_train = tf.data.Dataset.zip((dataset_train_x, dataset_train_y))
# dataset = dataset_train

def tfdata_generator(file_x, file_y, is_training, num_features = 54, SEQ_LEN = 60, batch_size=128):
    def preprocess_fn(features, label):
        x = tf.reshape(tf.cast(features, tf.float32), (1, num_features, SEQ_LEN))
        y = label
        return x, y
    
    filenames = [file_x, file_y]
    record_defaults_x = [tf.float32] * num_features*SEQ_LEN
    record_defaults_y = [tf.float32] * 1
    dataset_train_x = tf.contrib.data.CsvDataset(file_x,record_defaults_x)
    dataset_train_y = tf.contrib.data.CsvDataset(file_y,record_defaults_y)
    dataset = tf.data.Dataset.zip((dataset_train_x, dataset_train_y))
    if is_training:
        dataset = dataset.shuffle(1000)  # depends on sample size
        
    # Transform and batch data at the same time
    dataset = dataset.apply(tf.contrib.data.map_and_batch(
        preprocess_fn, batch_size,
        num_parallel_batches=4,  # cpu cores
        drop_remainder=True if is_training else False))
    dataset = dataset.repeat()
    dataset = dataset.prefetch(tf.contrib.data.AUTOTUNE)

    return dataset

In [22]:
dataset_train = tfdata_generator(file_train_x, file_train_y, is_training = True, num_features = 54, SEQ_LEN = 60, batch_size=128)
dataset_val = tfdata_generator(file_val_x, file_val_y, is_training = False, num_features = 54, SEQ_LEN = 60, batch_size=128)

  # model.fit(
  #     training_set.make_one_shot_iterator(),
  #     steps_per_epoch=len(x_train) // _BATCH_SIZE,
  #     epochs=_EPOCHS,
  #     validation_data=testing_set.make_one_shot_iterator(),
  #     validation_steps=len(x_test) // _BATCH_SIZE,
  #     verbose = 1)
    

In [26]:
next_element = dataset.make_one_shot_iterator().get_next()

with tf.Session() as sess:
    while True:
        try:
            x,y = sess.run(next_element)
            print(x)
            print("========")
            print(y)
            break
        except:
            print("TEST")



[[[[ 1.7249104e+00  4.1193125e-01  5.5222183e-01 ...  4.0652757e-03
     1.9495683e-02 -7.7644591e-03]
   [ 2.5082646e-02 -7.2491621e-03  1.0335121e-02 ...  2.9068461e-02
     2.1786433e-02  1.3424078e+00]
   [ 5.7113436e-03  6.5432340e-03 -1.2195255e-02 ...  2.4802674e-01
     5.3063464e-01  9.7802240e-01]
   ...
   [-2.5242031e-01  6.8401563e-01 -4.6024072e-01 ... -1.1841547e+00
     6.3488660e-03  8.4674436e-01]
   [ 1.0376310e+00  1.0376310e+00  7.0385462e-01 ... -5.7510769e-01
    -8.9733452e-01  1.3096903e-01]
   [-4.5506689e-01 -1.4080665e-02  8.1741011e-01 ... -1.2949303e+00
     0.0000000e+00 -6.1532289e-01]]]


 [[[-1.7920727e+00 -1.1475335e+00 -1.6546106e+00 ... -5.7773900e-01
    -2.4844501e+00 -4.2525420e+00]
   [-1.7637614e+00  6.8830550e-01 -3.5196528e-01 ...  3.6982900e-01
     3.3302224e-01 -9.2952130e-03]
   [ 1.2625822e+00  1.6122898e-01 -1.2195255e-02 ... -1.1276156e+00
     4.4618130e-01  9.9032646e-01]
   ...
   [ 6.2979293e-01 -3.3698918e-03 -1.1828543e-03 ... -1